<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
def get_vocabulary_components_from_corpus(corpus):
    splitted_corpus = np.char.split(corpus, ',')
    documents_in_corpus = [words.split(' ') for document in splitted_corpus
                              for words in document]
    vocabulary = set([word for document in documents_in_corpus
                              for word in document])
    vocabulary = list(vocabulary)
    return (sorted(vocabulary), splitted_corpus, documents_in_corpus)

In [5]:
vocabulary, _, _ = get_vocabulary_components_from_corpus(corpus)

In [6]:
print(vocabulary)

['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [7]:
def get_one_hot_encoder(corpus, add_column_names=True):
    vocabulary, splitted_corpus, documents_in_corpus = get_vocabulary_components_from_corpus(corpus)
    ohes = []
    if add_column_names:
        ohes.append(np.array(vocabulary, dtype=object))
    for document in documents_in_corpus:
        indexes = np.isin(vocabulary, document)
        ohe = np.zeros([len(vocabulary)])
        ohe[indexes] = 1
        ohes.append(np.array(ohe, dtype=int))
    return np.array(ohes)

In [8]:
ohe = get_one_hot_encoder(corpus, add_column_names=True)

In [9]:
print(ohe)

[['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']
 [0 1 0 1 0 1 0 0 1]
 [1 1 1 1 0 1 1 0 0]
 [0 0 0 0 1 0 1 1 0]]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [10]:
def get_count_vectorizer(corpus, add_column_names=True):
    vocabulary, splitted_corpus, documents_in_corpus = get_vocabulary_components_from_corpus(corpus)
    ohes = []
    if add_column_names:
        ohes.append(np.array(vocabulary, dtype=object))
    for document in documents_in_corpus:
        indexes = np.isin(vocabulary, document)
        indexes = np.where(indexes)
        ohe = np.zeros([len(vocabulary)])
        for index in indexes[0]:
            word = vocabulary[index]
            count = 0
            for w in document:
                if w == word:
                    count += 1
            ohe[index] = count
        ohes.append(np.array(ohe, dtype=int))
    return np.array(ohes)

In [11]:
cv = get_count_vectorizer(corpus, add_column_names=True)

In [12]:
print(cv)

[['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']
 [0 1 0 1 0 1 0 0 1]
 [1 1 1 1 0 1 2 0 0]
 [0 0 0 0 1 0 1 1 0]]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [13]:
def TF_IDF(corpus, add_column_names=True):
    vocabulary, splitted_corpus, _ = get_vocabulary_components_from_corpus(corpus)
    vocabulary = np.array(vocabulary, dtype=object)
    
    tf = get_count_vectorizer(corpus, add_column_names=False) 
    
    idf = get_one_hot_encoder(corpus, add_column_names=False)    
    num = len(splitted_corpus)
    den = np.sum(idf, axis=0)
    idf = np.log10(np.divide(num, den, where=den!=0))
    
    tf_idf = tf * idf
    
    if add_column_names:
        tf_idf = np.concatenate(([vocabulary], tf_idf), axis=0)

    return tf_idf

In [14]:
tf_idf = TF_IDF(corpus, add_column_names=True)

In [15]:
print(tf_idf)

[['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']
 [0.0 0.17609125905568124 0.0 0.17609125905568124 0.0 0.17609125905568124
  0.0 0.0 0.47712125471966244]
 [0.47712125471966244 0.17609125905568124 0.47712125471966244
  0.17609125905568124 0.0 0.17609125905568124 0.3521825181113625 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.47712125471966244 0.0 0.17609125905568124
  0.47712125471966244 0.0]]


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [16]:
def order_by_cosine_similarity(corpus, index, delete_item=True, use_tfidf=True):
    splitted_corpus = np.char.split(corpus, ',')
    if index < len(splitted_corpus):
        args = (corpus, False)
        elements = TF_IDF(*args) if use_tfidf else get_one_hot_encoder(*args)
        element = elements[index]
        scores = []
        for document in elements:
            cos_sim = cosine_similarity(document, element)
            scores.append(cos_sim)
        indexes = list(np.argsort(scores)[::-1])
        if delete_item:
            indexes.remove(index)
        return splitted_corpus[indexes]
    else:
        raise IndexError('Index out of range')

#### Referencias
- **delete_item**: indica si hay que eliminar de las respuestas ordenadas el item correspondiente al índice proporcionado para calcular la similaridad coseno.
- **use_tfidf**: si su valor es True utiliza el algoritmo TF IDF, en caso contrario utiliza el algoritmo One Hot Encoder para codificar los documentos.

In [17]:
print(corpus)

['que dia es hoy' 'martes el dia de hoy es martes' 'martes muchas gracias']


In [18]:
index = 0
print(corpus[index])

que dia es hoy


In [19]:
ordered = order_by_cosine_similarity(corpus, index)

In [20]:
print(ordered)

[list(['martes el dia de hoy es martes']) list(['martes muchas gracias'])]
